In [1]:
import nltk
from nltk.corpus import twitter_samples, stopwords
import pickle
import numpy as np
from package import process_tweets, cosine_similarity

In [2]:
positive_tweets = twitter_samples.strings('positive_tweets.json')
negative_tweets = twitter_samples.strings('negative_tweets.json')

all_tweets = positive_tweets + negative_tweets

In [3]:
en_embeddings = pickle.load(open('en_embeddings.p', 'rb')) #english word embeddings

In [4]:
def get_document_embedding(tweet, en_embeddings):
    
    doc_embedding = np.zeros(300)
    
    processed_doc = process_tweets(tweet)
    
    for word in processed_doc:
        doc_embedding += en_embeddings.get(word, 0)
    
    return doc_embedding

In [5]:
custom_tweet = "RT @Twitter @chapagain Hello There! Have a great day. :) #good #morning http://chapagain.com.np"

tweet_embedding = get_document_embedding(custom_tweet, en_embeddings)
tweet_embedding[-5:]

array([-0.00268555, -0.15378189, -0.55761719, -0.07216644, -0.32263184])

In [6]:
def get_document_vecs(docs, embeddings):
    
    doc_dict = {}
    
    doc_vecs = []
    
    for i, doc in enumerate(docs):
        doc_embedding = get_document_embedding(doc, embeddings)
        
        doc_dict[i] = doc_embedding
        
        doc_vecs.append(doc_embedding)
        
    doc_matrix = np.vstack(doc_vecs)
    
    return doc_matrix, doc_dict

In [7]:
doc_matrix, doc_dict = get_document_vecs(all_tweets, en_embeddings)

In [8]:
print("Length of dictionary: ", len(doc_dict))
print("Document vecs shape: ", doc_matrix.shape)

Length of dictionary:  10000
Document vecs shape:  (10000, 300)


In [35]:
my_tweet = "i am sad"
process_tweets(my_tweet)

['sad']

In [36]:
tweet_embedding = get_document_embedding(my_tweet, en_embeddings)

In [37]:
max_id = np.argmax(cosine_similarity(doc_matrix, tweet_embedding))
print(all_tweets[max_id])

@zoeeylim sad sad sad kid :( it's ok I help you watch the match HAHAHAHAHA


In [39]:
N_VECS = len(all_tweets) #no. of vectors
N_DIMS = len(doc_dict[1]) #dimension in each vector
print(N_VECS,N_DIMS)

10000 300


In [40]:
N_PLANES = 10 #no. of planes
N_UNIVERSES = 25 #no. of times to repeat the hashing

In [41]:
np.random.seed(11)
planes = [np.random.normal(size=(N_DIMS, N_PLANES)) for _ in range(N_UNIVERSES)]